# 加载并初始化

In [1]:
import sys, json
with open('config.json') as f:
    args = json.load(f)
sys.path.append('..')
PATH = '../RKD-SC/results/'

import torch
import torch.nn as nn
from tqdm.auto import tqdm 
from models import GenTeacher, GenStudentCNN, GenClassifier
from models.model import AttentionAutoEncoder
from utils import save_model, test_on_val
from utils.dataset import get_datasets
from utils.channels import Channels

device = args["global"]["device"]
teacher_model_name = args["model"]["teacher_model_name"]
student = GenStudentCNN(device)
teacher = GenTeacher(teacher_model_name, device)
classifier = GenClassifier(512, 100, [], device)
student.load_state_dict(torch.load('/home/ubuntu/users/dky/RKD-SC/results/distillation/source_encoder_kd.pt'))
classifier.load_state_dict(torch.load('/home/ubuntu/users/dky/RKD-SC/results/distillation/source_decoder_kd.pt'))

print(f"The number of parameters of student model is: {sum(p.numel() for p in student.parameters()) / 1e6:.2f}M" )
print(f"The number of parameters of teacher model is: {sum(p.numel() for p in teacher.parameters()) / 1e6:.2f}M" )

The number of parameters of student model is: 2.92M
The number of parameters of teacher model is: 87.85M


In [2]:
train_loader, test_loader = get_datasets(args["data"]["path"], args["data"]["batch_size"], args["data"]["dataset_name"])

# 蒸馏学生模型

In [3]:
teacher.eval()
student.train()
MIN_LOSS = 5
optimizer_stu = torch.optim.Adam(student.parameters(), lr=args["distillation"]["learning_rate"])
criterion_KD = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_stu, step_size=args["distillation"]["epochs"] // 2, gamma=0.1)

epoch_bar = tqdm(range(args["distillation"]["epochs"]), desc="Distillation")
for epoch in epoch_bar:
    batch_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch}", leave=False)
    for i, (data, _) in batch_bar:
        data = data.to(device)

        optimizer_stu.zero_grad()
        teacher_output = teacher(data)
        student_output = student(data)

        loss_KD = criterion_KD(student_output, teacher_output)
        batch_bar.set_postfix({'KD Loss': loss_KD.item(), 'LR': scheduler.get_last_lr()[0]})

        loss_KD.backward()
        optimizer_stu.step()

    scheduler.step()

    if loss_KD.item() < MIN_LOSS:
        MIN_LOSS = loss_KD.item()
        save_model(student, args["distillation"]["model_saved_path"] + 'source_encoder_kd.pt')

    epoch_bar.set_postfix({'KD Loss': loss_KD.item(), 'LR': scheduler.get_last_lr()[0]})

Distillation:   0%|          | 0/300 [00:00<?, ?it/s]

Epoch 0:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 3:   0%|          | 0/500 [00:00<?, ?it/s]

...

Epoch 295:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 296:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 297:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 298:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 299:   0%|          | 0/500 [00:00<?, ?it/s]

# 训练学生模型对应的分类网络

In [ ]:
classifier = GenClassifier(512, 100, [], device)
classifier.train()
student.eval()

optimizer_cla = torch.optim.Adam(classifier.parameters(), lr=args["classification"]["learning_rate"])
criterion_CE = nn.CrossEntropyLoss()

for epoch in tqdm(range(args["classification"]["epochs"]), desc="Epoch"):
    with tqdm(enumerate(train_loader),
              total=len(train_loader),
              desc=f"Epoch {epoch+1} Batch",
              leave=False) as batch_bar:
        for i, (data, target) in batch_bar:
            data, target = data.to(device), target.to(device)

            optimizer_cla.zero_grad()
            student_output = student(data)
            output = classifier(student_output)

            loss_CE = criterion_CE(output, target)
            acc = (output.argmax(dim=1) == target).float().mean()
            batch_bar.set_postfix({'CE Loss': loss_CE.item(), 'Accuracy': acc.item()})

            loss_CE.backward()
            optimizer_cla.step()

    save_model(classifier, args["distillation"]["model_saved_path"] + 'source_decoder_kd.pt')

acc_test = test_on_val(student, classifier, test_loader, device)

Testing on val... 	Accuracy:  0.6836


In [4]:
atten_ae = AttentionAutoEncoder(512, 2).to(device)
channel = Channels(device)
optimizer_cla = torch.optim.Adam(classifier.parameters(), lr=1e-4)
optimizer_ae = torch.optim.Adam(atten_ae.parameters(), lr=1e-4)
optimizer_stu = torch.optim.Adam(student.parameters(), lr=1e-4)

criterion_AE = torch.nn.MSELoss()
criterion_CE = torch.nn.CrossEntropyLoss()

student.train()
classifier.train()
atten_ae.train()

TRAIN_SNR_LIST = [-5, -2, 0, 5, 10, 15, 20]

for epoch, snr in enumerate(tqdm(TRAIN_SNR_LIST, desc="Epoch")):
    batch_bar = tqdm(train_loader,
                     leave=False,
                     desc=f"Epoch {epoch+1} Training on SNR={snr}")
    
    for i, (images, labels) in enumerate(batch_bar):
        images, labels = images.to(device), labels.to(device)

        optimizer_cla.zero_grad()
        optimizer_ae.zero_grad()
        optimizer_stu.zero_grad()

        features_stu = student(images)
        features_tea = teacher(images)

        latten_ae = atten_ae.encoder(features_stu)
        latten_ae = channel.AWGN(latten_ae, snr)
        features_hat = atten_ae.decoder(latten_ae)

        pre = classifier(features_hat)

        loss_AE = criterion_AE(features_hat, features_tea)
        loss_CE = criterion_CE(pre, labels)
        loss = loss_AE + loss_CE
        
        acc = (pre.argmax(dim=1) == labels).float().mean()

        loss.backward()

        optimizer_cla.step()
        optimizer_ae.step()
        optimizer_stu.step()

        batch_bar.set_postfix(loss=loss_AE.item(), acc=acc.item())

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1 Training on SNR=-5:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 2 Training on SNR=-2:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 3 Training on SNR=0:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 4 Training on SNR=5:   0%|          | 0/500 [00:00<?, ?it/s]

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe2d423d280>
Traceback (most recent call last):
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/home/ubuntu/users/dky/anaconda3/envs/KD/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe2d423d280>
Traceback (most recent call last):
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packa

Epoch 6 Training on SNR=15:   0%|          | 0/500 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe2d423d280>
Traceback (most recent call last):
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/home/ubuntu/users/dky/anaconda3/envs/KD/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe2d423d280>
Traceback (most recent call last):
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packa

Epoch 7 Training on SNR=20:   0%|          | 0/500 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe2d423d280>
Traceback (most recent call last):
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/home/ubuntu/users/dky/anaconda3/envs/KD/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe2d423d280>
Traceback (most recent call last):
  File "/home/ubuntu/users/dky/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe2d423d280>  

In [5]:
student.eval()
classifier.eval()
atten_ae.eval()
channels = Channels(device)

ACC_with_SNR = []

outer_bar = tqdm(range(-10, 25, 2), desc='Testing over SNR')
for snr in outer_bar:

    batch_bar = tqdm(test_loader, desc=f"Testing on SNR: {snr}", leave=False)
    total_correct = 0
    total_samples = 0
    
    for images, labels in batch_bar:
        images, labels = images.to(device), labels.to(device)
        
        with torch.no_grad():
            features = student(images)
            features = atten_ae.encoder(features)
            features_awgn = channels.AWGN(features, snr)
            features_awgn = atten_ae.decoder(features_awgn)
            logits = classifier(features_awgn)
            preds = logits.argmax(dim=1)
            
            total_correct += (preds == labels).float().sum().item()
            total_samples += labels.size(0)
        
        batch_bar.set_postfix({'Acc': total_correct / total_samples})

    acc = total_correct / total_samples
    ACC_with_SNR.append(acc)
    outer_bar.set_postfix({'SNR': snr, 'Acc': acc})

print(ACC_with_SNR)

Testing over SNR:   0%|          | 0/18 [00:00<?, ?it/s]

Testing on SNR: -10:   0%|          | 0/100 [00:00<?, ?it/s]

Testing on SNR: -8:   0%|          | 0/100 [00:00<?, ?it/s]

Testing on SNR: -6:   0%|          | 0/100 [00:00<?, ?it/s]

Testing on SNR: -4:   0%|          | 0/100 [00:00<?, ?it/s]

...

Testing on SNR: 18:   0%|          | 0/100 [00:00<?, ?it/s]

Testing on SNR: 20:   0%|          | 0/100 [00:00<?, ?it/s]

Testing on SNR: 22:   0%|          | 0/100 [00:00<?, ?it/s]

Testing on SNR: 24:   0%|          | 0/100 [00:00<?, ?it/s]

[0.2676, 0.3524, 0.4505, 0.5205, 0.5795, 0.6251, 0.6505, 0.6761, 0.6925, 0.6981, 0.7014, 0.7058, 0.71, 0.7106, 0.7098, 0.71, 0.7109, 0.7132]
